<a href="https://colab.research.google.com/github/FernandoAltamirano/sistemas-inteligentes/blob/main/ModeloEstandarizadoCapaAdicional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO ESTANDARIZADO CON CAPA ADICIONAL

In [1]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
dataframe = read_csv("ionosphere.csv",header=None)

In [3]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [4]:
dataset = dataframe.values

In [5]:
X = dataset[:,0:34].astype(float)

In [6]:
Y = dataset[:,34]

In [7]:
codificador = LabelEncoder()
codificador.fit(Y)
Y_codificado = codificador.transform(Y)

In [8]:
def modelo_base():
    # arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(65,input_dim=34,activation='relu'))
    modelo.add(Dense(35,activation='relu'))
    modelo.add(Dense(1,activation='sigmoid'))
    # compila el modelo
    modelo.compile(loss='binary_crossentropy',
    optimizer='adam',metrics=['accuracy'])
    
    return modelo

In [9]:
estimadores = []

In [10]:
estimadores.append(('estandarizacion',StandardScaler()))

In [11]:
estimadores.append(('red_neuronal',KerasClassifier(build_fn=modelo_base,epochs=120,batch_size=5,verbose=0)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [12]:
canal = Pipeline(estimadores)

In [13]:
kfold = StratifiedKFold(n_splits=10,shuffle=True)

In [14]:
resultados = cross_val_score(canal,X,Y_codificado,cv=kfold)

In [15]:
resultados

array([0.97222221, 0.91428572, 0.91428572, 0.91428572, 0.97142857,
       0.94285715, 0.97142857, 0.94285715, 0.88571429, 0.91428572])

In [16]:
print("Accuracy: %.2f%% (%.2f%%)" % (resultados.mean()*100,resultados.std()*100))

Accuracy: 93.44% (2.88%)
